<a href="https://colab.research.google.com/github/joaogabrielanuto/quant-traders/blob/under-construction/detector-topos-fundos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://tradingcomdados.com/images/logotipo/logotipo-trading-com-dados.svg" width="300" align="left"/>

---
# **Detector de topos e fundos** 


#### Metodologia utilizando mínimas e máximas do candles
---

# 1. Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import yfinance as yf
import mplfinance as mpf
import plotly.graph_objects as go

# 2. Entendendo a metodologia

Obtenção de dados OHLC

In [ ]:
df = yf.download('^BVSP', start='2022-01-01', end='2023-01-01')
mpf.plot(df[-30:], type='candle')

# 3. Construção das funções

## 3.1. Primeira etapa

Vamos retomar os dados diários do IBOV em 2022 e checar o **candle do dia 27/dez, que é um fundo segundo nossa metodologia para 2 candles**

Note que que o candle referência atende às 2 regras:
* Mínima (candle referência) é menor que a dos dois candles anteriores,
* Mínima (candle referência) é menor que a dos dois candles posteriores

In [ ]:
mpf.plot(df[-7:], type='candle')

In [ ]:
mpf.plot(df[:10], type='candle')

In [ ]:
df['Low'] <= df['Low'].rolling(2, closed='left').min()

In [ ]:
df_invertido = df.iloc[::-1]
df_invertido.head()

In [ ]:
df_invertido = df.iloc[::-1]
df_invertido['Low'] <= df_invertido['Low'].rolling(2, closed='left').min()

In [ ]:
def localizar_fundos(df, n_candles=2):
    
    df_invertido = df.iloc[::-1]

    cond_low_anteriores = df['Low'] <= df['Low'].rolling(n_candles, closed='left').min()
    cond_low_posteriores = df_invertido['Low'] <= df_invertido['Low'].rolling(n_candles, closed='left').min()

    return (cond_low_anteriores & cond_low_posteriores)

In [ ]:
localizar_fundos(df)

In [ ]:
def localizar_topos(df, n_candles=2):
    
    df_invertido = df.iloc[::-1]

    cond_high_anteriores = df['High'] >= df['High'].rolling(n_candles, closed='left').max()
    cond_high_posteriores = df_invertido['High'] >= df_invertido['High'].rolling(n_candles, closed='left').max()
  
    return (cond_high_anteriores & cond_high_posteriores)

In [ ]:
localizar_topos(df)

In [ ]:
n_candles = 2

dict_topos_fundos = {'vertice':[], 'data':[], 'preco':[]}

for i in range(len(df)):

    if localizar_fundos(df, n_candles).iloc[i]:
        dict_topos_fundos['vertice'].append('fundo')
        dict_topos_fundos['data'].append(df.index[i])
        dict_topos_fundos['preco'].append(df.Low.iloc[i])
    
    if localizar_topos(df, n_candles).iloc[i]:
        dict_topos_fundos['vertice'].append('topo')
        dict_topos_fundos['data'].append(df.index[i])
        dict_topos_fundos['preco'].append(df.High.iloc[i])

In [ ]:
df_topos_fundos = pd.DataFrame(dict_topos_fundos)
df_topos_fundos.index = df_topos_fundos.data
df_topos_fundos.head(10)

In [ ]:
df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')
df_final.head(10)

In [ ]:
df_figure = df_final

fig_topos_fundos = go.Figure(data=[go.Candlestick(name='', x=df_figure.index, open=df_figure.Open, high = df_figure.High,
                                                  low=df_figure.Low, close=df_figure.Close, 
                                                  increasing_line_color= 'green', decreasing_line_color= 'red',
                                                  showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(x=df_topos_fundos.data, y=df_topos_fundos.preco,
                                      mode="lines", line_width=2, line_dash="solid", line_color="black",
                                      name="Topos / Fundos", showlegend=True))

fig_topos_fundos.update_xaxes(rangebreaks=[
                              dict(bounds=["sat", "mon"])])

fig_topos_fundos.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Indicador topos e fundos',
                               template = 'none', margin=dict(l=75, r=75, t=75, b=75), 
                               paper_bgcolor="#f7f8fa",
                               width=1200,height=600)

## 3.2. Segunda etapa

In [ ]:
n_candles = 2
ls_vertices = ['neutro']

dict_topos_fundos = {'vertice':[], 'data':[], 'preco':[]}

for i in range(len(df)):

    if localizar_fundos(df, n_candles).iloc[i]:

        if ls_vertices[-1]!='fundo':
            dict_topos_fundos['vertice'].append('fundo')
            dict_topos_fundos['data'].append(df.index[i])
            dict_topos_fundos['preco'].append(df.Low.iloc[i])
            ls_vertices.append('fundo')
        
        else:
            if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
                dict_topos_fundos['data'][-1] = df.index[i]
                dict_topos_fundos['preco'][-1] = df.Low.iloc[i]
    
    if localizar_topos(df, n_candles).iloc[i]:

        if ls_vertices[-1]!='topo':
            dict_topos_fundos['vertice'].append('topo')
            dict_topos_fundos['data'].append(df.index[i])
            dict_topos_fundos['preco'].append(df.High.iloc[i])
            ls_vertices.append('topo')

        else:
            if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
                dict_topos_fundos['data'][-1] = df.index[i]
                dict_topos_fundos['preco'][-1] = df.High.iloc[i]

In [ ]:
df_topos_fundos = pd.DataFrame(dict_topos_fundos)
df_topos_fundos.index = df_topos_fundos.data

df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')
df_final.head(20)

df_figure = df_final

fig_topos_fundos = go.Figure(data=[go.Candlestick(name='', x=df_figure.index, open=df_figure.Open, high = df_figure.High,
                                                  low=df_figure.Low, close=df_figure.Close, 
                                                  increasing_line_color= 'green', decreasing_line_color= 'red',
                                                  showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(x=df_topos_fundos.data, y=df_topos_fundos.preco,
                                      mode="lines", line_width=2, line_dash="solid", line_color="black",
                                      name="Topos / Fundos", showlegend=True))

fig_topos_fundos.update_xaxes(rangebreaks=[
                              dict(bounds=["sat", "mon"])])

fig_topos_fundos.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Indicador topos e fundos',
                               template = 'none', margin=dict(l=75, r=75, t=75, b=75), 
                               paper_bgcolor="#f7f8fa",
                               width=1200,height=600)

# 4. Teoria de Dow

In [ ]:
df_1w = yf.download('^BVSP', start='2020-01-01', end='2023-01-01', interval='1wk')
df_1d = yf.download('^BVSP', start='2020-01-01', end='2023-01-01', interval='1d')

In [ ]:
df = df_1w.copy()

n_candles = 10
ls_vertices = ['neutro']

dict_topos_fundos = {'vertice':[], 'data':[], 'preco':[]}

for i in range(len(df)):

    if localizar_fundos(df, n_candles).iloc[i]:

        if ls_vertices[-1]!='fundo':
            dict_topos_fundos['vertice'].append('fundo')
            dict_topos_fundos['data'].append(df.index[i])
            dict_topos_fundos['preco'].append(df.Low.iloc[i])
            ls_vertices.append('fundo')
        
        else:
            if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
                dict_topos_fundos['data'][-1] = df.index[i]
                dict_topos_fundos['preco'][-1] = df.Low.iloc[i]
    
    if localizar_topos(df, n_candles).iloc[i]:

        if ls_vertices[-1]!='topo':
            dict_topos_fundos['vertice'].append('topo')
            dict_topos_fundos['data'].append(df.index[i])
            dict_topos_fundos['preco'].append(df.High.iloc[i])
            ls_vertices.append('topo')

        else:
            if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
                dict_topos_fundos['data'][-1] = df.index[i]
                dict_topos_fundos['preco'][-1] = df.High.iloc[i]

df_topos_fundos = pd.DataFrame(dict_topos_fundos)
df_topos_fundos.index = df_topos_fundos.data

df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')
df_final.head(20)

df_figure = df_final

fig_topos_fundos = go.Figure(data=[go.Candlestick(name='', x=df_figure.index, open=df_figure.Open, high = df_figure.High,
                                                  low=df_figure.Low, close=df_figure.Close, 
                                                  increasing_line_color= 'green', decreasing_line_color= 'red',
                                                  showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(x=df_topos_fundos.data, y=df_topos_fundos.preco,
                                      mode="lines", line_width=2, line_dash="solid", line_color="black",
                                      name="Topos / Fundos", showlegend=True))

fig_semanal = fig_topos_fundos.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Indicador topos e fundos',
                               template = 'none', margin=dict(l=75, r=75, t=75, b=75), 
                               paper_bgcolor="#f7f8fa",
                               width=1200,height=600)

In [ ]:
df = df_1d.copy()

n_candles = 10
ls_vertices = ['neutro']

dict_topos_fundos = {'vertice':[], 'data':[], 'preco':[]}

for i in range(len(df)):

    if localizar_fundos(df, n_candles).iloc[i]:

        if ls_vertices[-1]!='fundo':
            dict_topos_fundos['vertice'].append('fundo')
            dict_topos_fundos['data'].append(df.index[i])
            dict_topos_fundos['preco'].append(df.Low.iloc[i])
            ls_vertices.append('fundo')
        
        else:
            if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
                dict_topos_fundos['data'][-1] = df.index[i]
                dict_topos_fundos['preco'][-1] = df.Low.iloc[i]
    
    if localizar_topos(df, n_candles).iloc[i]:

        if ls_vertices[-1]!='topo':
            dict_topos_fundos['vertice'].append('topo')
            dict_topos_fundos['data'].append(df.index[i])
            dict_topos_fundos['preco'].append(df.High.iloc[i])
            ls_vertices.append('topo')

        else:
            if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
                dict_topos_fundos['data'][-1] = df.index[i]
                dict_topos_fundos['preco'][-1] = df.High.iloc[i]

df_topos_fundos = pd.DataFrame(dict_topos_fundos)
df_topos_fundos.index = df_topos_fundos.data

df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')
df_final.head(20)

df_figure = df_final

fig_topos_fundos = go.Figure(data=[go.Candlestick(name='', x=df_figure.index, open=df_figure.Open, high = df_figure.High,
                                                  low=df_figure.Low, close=df_figure.Close, 
                                                  increasing_line_color= 'green', decreasing_line_color= 'red',
                                                  showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(x=df_topos_fundos.data, y=df_topos_fundos.preco,
                                      mode="lines", line_width=2, line_dash="solid", line_color="black",
                                      name="Topos / Fundos", showlegend=True))

fig_diario = fig_topos_fundos.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Indicador topos e fundos',
                               template = 'none', margin=dict(l=75, r=75, t=75, b=75), 
                               paper_bgcolor="#f7f8fa",
                               width=1200,height=600)

In [ ]:
fig_semanal

In [ ]:
fig_diario

# 5. Screening ativos

In [ ]:
lista_tickers = ['ABEV3', 'ALPA4', 'ALSO3', 'ARZZ3', 'ASAI3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4',
                'BBSE3', 'BEEF3', 'BPAC11', 'BPAN4', 'BRAP4', 'BRFS3', 'BRKM5', 'CASH3', 'CCRO3', 'CIEL3', 'CMIG4', 'CMIN3',
                'COGN3', 'CPFE3', 'CPLE6', 'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'DXCO3', 'ECOR3', 'EGIE3', 'ELET3',
                'ELET6', 'EMBR3', 'ENBR3', 'ENEV3', 'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'HAPV3',
                'HYPE3', 'IGTI11', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3', 'LWSA3', 'MGLU3', 'MRFG3', 'MRVE3', 'MULT3',
                'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PETZ3', 'PRIO3', 'QUAL3', 'RADL3', 'RAIL3', 'RAIZ4', 'RDOR3', 'RENT3',
                'RRRP3', 'SANB11', 'SBSP3', 'SLCE3', 'SMTO3', 'SOMA3', 'SUZB3', 'TAEE11', 'TIMS3', 'TOTS3', 'UGPA3', 'USIM5',
                'VALE3', 'VBBR3', 'VIIA3', 'VIVT3', 'WEGE3', 'YDUQ3']

In [ ]:
lista_tickers_yf = [x + '.SA' for x in lista_tickers[2:]]

In [ ]:
lista_tickers_yf[0:5]

In [ ]:
ls_close_ativos = []
indice_ativo = 0

for i in lista_tickers_yf:
    df_1d_ativo = yf.download(i, start='2022-01-01', interval='1d', auto_adjust=True)
    df_1d_ativo['Ticker'] = i
    df_1d_ativo['Indice_ativo'] = indice_ativo
    indice_ativo +=1
    ls_close_ativos.append(df_1d_ativo)

In [ ]:
list(enumerate(lista_tickers_yf))

In [ ]:
ls_close_ativos[1]

In [ ]:
n_candles = 10

ls_ativos_topos_fundos = []

for df in ls_close_ativos:
    
    ls_vertices = ['neutro']
    dict_topos_fundos = {'vertice':[], 'data':[], 'preco':[]}

    for i in range(len(df)):

        if localizar_fundos(df, n_candles).iloc[i]:

            if ls_vertices[-1]!='fundo':
                dict_topos_fundos['vertice'].append('fundo')
                dict_topos_fundos['data'].append(df.index[i])
                dict_topos_fundos['preco'].append(df.Low.iloc[i])
                ls_vertices.append('fundo')
            
            else:
                if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
                    dict_topos_fundos['data'][-1] = df.index[i]
                    dict_topos_fundos['preco'][-1] = df.Low.iloc[i]
        
        if localizar_topos(df, n_candles).iloc[i]:

            if ls_vertices[-1]!='topo':
                dict_topos_fundos['vertice'].append('topo')
                dict_topos_fundos['data'].append(df.index[i])
                dict_topos_fundos['preco'].append(df.High.iloc[i])
                ls_vertices.append('topo')

            else:
                if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
                    dict_topos_fundos['data'][-1] = df.index[i]
                    dict_topos_fundos['preco'][-1] = df.High.iloc[i]
    
    df_topos_fundos = pd.DataFrame(dict_topos_fundos)
    df_topos_fundos.index = df_topos_fundos.data

    df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')
    df_final.head(20)

    ls_ativos_topos_fundos.append(df_final)

In [ ]:
def ativo_rompeu_topo(df):
    ultimo_topo = df[df.vertice=='topo']['preco'][-1]
    distancia_topo = (ultimo_topo - df.Close[-1])/ultimo_topo*100
    
    if distancia_topo < 0:
        return 'rompeu topo'

In [ ]:
for df in ls_ativos_topos_fundos:

    if ativo_rompeu_topo(df)=='rompeu topo':
        print(df.Ticker[0],'indice=',df.Indice_ativo[0],'rompeu topo anterior')

In [ ]:
i = 4

df_figure = ls_ativos_topos_fundos[i]
df_topos_fundos = ls_ativos_topos_fundos[i][~ls_ativos_topos_fundos[i].preco.isna()]

fig_topos_fundos = go.Figure(data=[go.Candlestick(name='', x=df_figure.index, open=df_figure.Open, high = df_figure.High,
                                                  low=df_figure.Low, close=df_figure.Close, 
                                                  increasing_line_color= 'green', decreasing_line_color= 'red',
                                                  showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(x=df_topos_fundos.index, y=df_topos_fundos.preco,
                                      mode="lines", line_width=2, line_dash="solid", line_color="black",
                                      name="Topos / Fundos", showlegend=True))

fig_topos_fundos.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Indicador topos e fundos',
                               template = 'none', margin=dict(l=75, r=75, t=75, b=75), 
                               paper_bgcolor="#f7f8fa",
                               width=1200,height=600)